In [2]:
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


print(tf.__version__)

2.3.1


In [3]:
url="http://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"

dataset = tf.keras.utils.get_file("stack_overflow_16k.tar.gz", url, untar=True, cache_dir='.', cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'stack_overflow')

6053888/6053168 [==============================] - 2s 0us/step


In [6]:
print(os.listdir(dataset_dir))

FileNotFoundError: [WinError 3] The system cannot find the path specified: '.\\stack_overflow_16k.tar.gz'